In [1]:
%config Completer.use_jedi = False

In [2]:
from azureml.core import Workspace, Experiment
from azureml.core.compute_target import ComputeTargetException

ws = Workspace.from_config()
print("Workspace: "+ ws.name, "Region: ")

Workspace: quick-starts-ws-156363 Region: 


In [3]:

#ws = Workspace.get(name="quick-starts-ws-156029")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')
exp = Experiment(workspace=ws, name="HyperDrive-Experiment-156029")

run = exp.start_logging()

Workspace name: quick-starts-ws-156363
Azure region: southcentralus
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-156363


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                              max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress.....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive import BanditPolicy

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C" : uniform(0.1,1),
    "--max_iter":choice(50,100,150,200)})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)
#BanditPolicy(evaluation_interval=1, **slack_factor=0.01,** delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./", compute_target = cpu_cluster, vm_size = "STANDARD_D2_V2", entry_script = "train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
#hyperdrive_config = HyperDriveConfig(estimator=est, hyperparameter_sampling=ps, primary_metric_name='Accuracy', primary_metric_goal=PrimaryMetr., max_total_runs=18, max_concurrent_runs=4 policy=policy,)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [7]:
primary_metric_goal = PrimaryMetricGoal.MAXIMIZE
primary_metric_name='Accuracy'
max_total_runs = 10
estimators = 20

hyperdrive_run_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name=primary_metric_name,
    primary_metric_goal=primary_metric_goal,
    max_total_runs=max_total_runs,
    max_concurrent_runs=4)

In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
exp = Experiment(workspace=ws, name='hypertuning')
hyperdrive_run = exp.submit(hyperdrive_run_config)
hyperdrive_run.get_status()
RunDetails(hyperdrive_run).show()
#hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [28]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics =  best_run.get_metrics()
parameter_values = best_run.get_details()#['runDefinition','Arguments']
#print(parameter_values)
print(best_run.get_details()['runDefinition']['arguments'])
print("Best Run ID: ",best_run.id)
joblib.dump(value=best_run.id, filename="./outputs/best_hyperdrive_model.joblib")

['--C', '0.10121952894476488', '--max_iter', '100']
Best Run ID:  HD_23677386-469f-4193-8c5b-6d73676061a4_1


['./outputs/best_hyperdrive_model.joblib']

In [14]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
df = ds.to_pandas_dataframe()
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [16]:
!python train.py --C=0.1 --max_iter=5

Attempted to log scalar metric Regularization Strength::
0.1
Attempted to log scalar metric Max iterations::
5
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
Attempted to log scalar metric Accuracy:
0.887556904400607


In [24]:
from train import clean_data
from sklearn.model_selection import train_test_split
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
#x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=42)
#x_train.head()
x.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_cellular,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown
0,57,1,0,0,1,5,1,371,1,999,...,1,0,0,0,0,1,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,0,1,0,0,0,0,0,0,0,1
2,33,1,0,0,0,5,5,52,1,999,...,1,0,0,0,1,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,0,1,0,0,0,1,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,1,0,0,0,0,1,0,0,0,0


In [25]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.


automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=df,
    label_column_name='y',
    n_cross_validations=2)

In [27]:
# Submit your automl run

### YOUR CODE HERE ###
exp = Experiment(workspace=ws, name="automl-logisticregression")
remote_run = exp.submit(automl_config,show_output=False)
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
automl-logisticregression,AutoML_1b460cb0-f7f1-4d68-b56f-4f85b1277e7c,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


Experiment,Id,Type,Status,Details Page,Docs Page
automl-logisticregression,AutoML_1b460cb0-f7f1-4d68-b56f-4f85b1277e7c,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|3692                             |yes                              |32950                                 |
+---------------------------------+----------

{'runId': 'AutoML_1b460cb0-f7f1-4d68-b56f-4f85b1277e7c',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-08-31T01:48:59.464764Z',
 'endTimeUtc': '2021-08-31T02:15:14.168415Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-logisticregression","subscription_id":"510b94ba-e453-4417-988b-fbdc37b55ca7","resource_group":"aml-quickstarts-156363","workspace_name":"quick-starts-ws-156363","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metr

In [29]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)
joblib.dump(value=best_run.id, filename="./outputs/best_auto_model.joblib")

Run(Experiment: automl-logisticregression,
Id: AutoML_1b460cb0-f7f1-4d68-b56f-4f85b1277e7c_37,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=0, reg_alpha=0.625, reg_lambda=0.20833333333333334, subsample=1, tree_method='auto'))], verbose=False))], flatten_transform=None, weights=[0.13333333333333333, 0.13333333333333333, 0.06666666666666667, 0.06666666666666667, 0.26666666666666666, 0.06666666666666667, 0.2, 0.06666666666666667]))],
         verbose=False)
Y_transformer(['LabelEncoder', LabelEncoder()])


['./outputs/best_auto_model.joblib']

In [18]:
# Deleting Compute Cluster

cpu_cluster.delete()